In [1]:
import os, os.path
os.chdir('../')

In [2]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [3]:
import jieba 
import jieba.analyse

### Read the Data

In [4]:
data = pd.read_csv('data/processed_data.csv')

In [5]:
data.head()

,Unnamed: 0,id,text1,text2,label
0,0,1,﻿怎么更改花呗手机号码,我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号,1
1,1,2,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0
2,2,3,花呗冻结以后还能开通吗,我的条件可以开通花呗借款吗,0
3,3,4,如何得知关闭借呗,想永久关闭借呗,0
4,4,5,花呗扫码付钱,二维码扫描可以用花呗吗,0


### Vectorization

In [5]:
text1 = list(data['text1'])
text2 = list(data['text2'])
labels = list(data['label'])
assert len(text1) == len(text2)
texts = text1 + text2

In [6]:
tokens = []
for text in texts:
    for sentence in re.findall(r'\w+', text):
        for i in range(len(sentence)-1):
            word = sentence[i:i+2]
            tokens.append(word)
counter_2 = Counter(tokens)
most_common_2 = [word[0] for word in counter_2.most_common(30)]

In [7]:
tokens_3 = []
texts = data['text1']
for text in texts:
    for sentence in re.findall(r'\w+', text):
        for i in range(len(sentence)-2):
            word = sentence[i:i+3]
            tokens_3.append(word)
            
counter_3 = Counter(tokens_3)
most_common_3 = [word[0] for word in counter_3.most_common(30)]

In [8]:
most_common_2

['花呗',
 '借呗',
 '怎么',
 '什么',
 '还款',
 '可以',
 '蚂蚁',
 '为什',
 '额度',
 '我的',
 '蚁借',
 '分期',
 '开通',
 '用花',
 '支付',
 '的花',
 '呗还',
 '不能',
 '没有',
 '使用',
 '呗分',
 '付款',
 '呗的',
 '不了',
 '付宝',
 '个月',
 '呗怎',
 '蚁花',
 '么我',
 '呗额']

In [9]:
word = most_common_2[0]
word

'花呗'

In [10]:
keywords = most_common_2 + most_common_3
word_vector = np.zeros((len(text1), 2*len(keywords)))
for i, word in enumerate(keywords):
    ip = i + len(keywords)
    for j in range(len(word_vector)):
        if word in text1[j]:
            word_vector[j, i] = 1
        if word in text2[j]:
            word_vector[j, ip] = 1

## Key word extraction

In [11]:
def get_key_word(sentence,topK):
    return jieba.analyse.extract_tags(sentence,topK = topK)
def get_all_key_word(sentences,topK):
    res = []
    for i in range(len(sentences)):
        key_word = get_key_word(sentences[i],topK)
        res.extend(key_word)
    return res

In [12]:
# add dictionary words
added_words = ['花呗','借呗','怎么','什么','我的']
for i in added_words:
    jieba.add_word(i)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.991 seconds.
Prefix dict has been built succesfully.


In [13]:
counter_key = Counter(get_all_key_word(text1,5)+get_all_key_word(text2,5))

In [14]:
most_common_key = [word[0] for word in counter_key.most_common(120)]

In [15]:
all_key = [word for word in counter_key] #7000+ 

In [18]:
#### get training data

word_vector = np.zeros((len(text1), 2*len(most_common_key)))
for i, word in enumerate(most_common_key):
    ip = i + len(most_common_key)
    for j in range(len(word_vector)):
        if word in text1[j]:
            word_vector[j, i] = 1
        if word in text2[j]:
            word_vector[j, ip] = 1

## Training

In [19]:
data = pd.DataFrame(word_vector)

In [20]:
input_dim = word_vector.shape[1]

In [21]:
data_ = np.array(data)
labels = np.array(labels)

In [46]:
model = Sequential()
model.add(Dense(64, input_dim=input_dim, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(data_, labels, epochs=15, batch_size=10)
scores = model.evaluate(data_, labels)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/150
39312/39312 [==============================] - 6s 163us/step - loss: 0.5196 - acc: 0.7824
Epoch 2/150
39312/39312 [==============================] - 5s 121us/step - loss: 0.5097 - acc: 0.7826
Epoch 3/150
39312/39312 [==============================] - 5s 117us/step - loss: 0.5027 - acc: 0.7840
Epoch 4/150
39312/39312 [==============================] - 5s 120us/step - loss: 0.4948 - acc: 0.7843
Epoch 5/150
39312/39312 [==============================] - 5s 120us/step - loss: 0.4894 - acc: 0.7854
Epoch 6/150
39312/39312 [==============================] - 5s 134us/step - loss: 0.4849 - acc: 0.7855
Epoch 7/150
39312/39312 [==============================] - 5s 128us/step - loss: 0.4809 - acc: 0.7865
Epoch 8/150
39312/39312 [==============================] - 5s 126us/step - loss: 0.4767 - acc: 0.7879
Epoch 9/150
39312/39312 [==============================] - 5s 121us/step - loss: 0.4727 - acc: 0.7879
Epoch 10/150
39312/39312 [==============================] - 5s 120us/step - loss: 